In [24]:
import os
import sys
import pandas as pd
import pickle

import torch
from torch.utils.data import Dataset, Subset

sys.path.append(os.path.dirname(os.getcwd()))
print(os.getcwd())

from EDP_Model_Testing.test_lstm import WindTurbineDataset, get_aggregate_data

/sfs/gpfs/tardis/home/ujx4ab/ondemand/dissecting_dist_inf/WF_Data/EDP/EDP_Model_Testing


In [16]:
def get_data_splits(dataset, split_timestamp, batch_size=32):
    if isinstance(split_timestamp, str):
        split_timestamp = pd.Timestamp(split_timestamp)
        
    indices = list(range(len(dataset)))
    timestamps = dataset.timestamps

    train_val_indices = [i for i, t in zip(indices, timestamps) if t < split_timestamp]
    test_indices = [i for i, t in zip(indices, timestamps) if t >= split_timestamp]

    test_dataset = Subset(dataset, test_indices)
    train_val_dataset = Subset(dataset, train_val_indices)

    return test_dataset, train_val_dataset

In [17]:
DATA_FOLDER = "/home/ujx4ab/ondemand/dissecting_dist_inf/WF_Data/EDP/EDP_Model_Testing/data_prep"
FAULTS_DATA_FOLDER = DATA_FOLDER + "_faults_included"

X_tensor, y_tensor, timestamps_array = get_aggregate_data(DATA_FOLDER)
full_dataset = WindTurbineDataset(
    X=X_tensor,
    y=y_tensor,
    timestamps=timestamps_array
)   

faults_X_tensor, faults_y_tensor, faults_timestamps_array = get_aggregate_data(FAULTS_DATA_FOLDER)
faults_full_dataset = WindTurbineDataset(
    X=X_tensor,
    y=y_tensor,
    timestamps=timestamps_array
)   

Aggregating data ...
EDP_WT_01.csv
EDP_WT_01_X.pt
EDP_WT_01_timestamp.pkl
EDP_WT_01_y.pt
EDP_WT_06.csv
EDP_WT_06_X.pt
EDP_WT_06_timestamp.pkl
EDP_WT_06_y.pt
EDP_WT_07.csv
EDP_WT_07_X.pt
EDP_WT_07_timestamp.pkl
EDP_WT_07_y.pt
EDP_WT_11.csv
EDP_WT_11_X.pt
EDP_WT_11_timestamp.pkl
EDP_WT_11_y.pt
accumulated_timestamps.csv
X_tensor shape: torch.Size([420477, 145, 22])
y_tensor shape: torch.Size([420477, 1])
Number of timestamps: 420477
Aggregating data ...
EDP_WT_01.csv
EDP_WT_01_X.pt
EDP_WT_01_timestamp.pkl
EDP_WT_01_y.pt
EDP_WT_06.csv
EDP_WT_06_X.pt
EDP_WT_06_timestamp.pkl
EDP_WT_06_y.pt
EDP_WT_07.csv
EDP_WT_07_X.pt
EDP_WT_07_timestamp.pkl
EDP_WT_07_y.pt
EDP_WT_11.csv
EDP_WT_11_X.pt
EDP_WT_11_timestamp.pkl
EDP_WT_11_y.pt
X_tensor shape: torch.Size([420477, 145, 22])
y_tensor shape: torch.Size([420477, 1])
Number of timestamps: 420477


In [18]:
if (X_tensor.shape != faults_X_tensor.shape or 
    y_tensor.shape != faults_y_tensor.shape or 
    timestamps_array.shape != faults_timestamps_array.shape):
    print("Datasets have different shapes.")
else:
    if (torch.equal(X_tensor, faults_X_tensor) and
        torch.equal(y_tensor, faults_y_tensor) and
        (timestamps_array == faults_timestamps_array).all()):
        print("Datasets are identical.")
    else:
        print("Datasets are different.")
if not (timestamps_array == faults_timestamps_array).all():
    print(f"Unique timestamps in DATA_FOLDER: {set(timestamps_1)}")
    print(f"Unique timestamps in FAULTS_DATA_FOLDER: {set(timestamps_2)}")

Datasets are different.


In [20]:
_, train_val_dataset = get_data_splits(full_dataset, split_timestamp="2017-06-01")
faults_test_dataset, _ = get_data_splits(faults_full_dataset, split_timestamp="2017-06-01")

In [22]:
type(faults_test_dataset)

torch.utils.data.dataset.Subset

In [25]:
with open("/home/ujx4ab/ondemand/dissecting_dist_inf/datasets/LSTM_WTs/train.p", "wb") as f:
    pickle.dump(train_val_dataset, f)

with open("/home/ujx4ab/ondemand/dissecting_dist_inf/datasets/LSTM_WTs/test.p", "wb") as f:
    pickle.dump(faults_test_dataset, f)